In [2]:
!pip install scikeras

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.image import ImageDataGenerator

In [4]:
# Загрузка и предобработка данных
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [12]:
def get_model(hidden_layer_dim, meta):

    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(32, 32, 3)))
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

In [13]:
clf = KerasClassifier(
    get_model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
)

In [14]:
params = {
    "hidden_layer_dim": [50, 100, 200],
    "loss": ["sparse_categorical_crossentropy"],
    "optimizer": ["adam", "sgd"],
    "optimizer__learning_rate": [0.0001, 0.001, 0.1],
}
gs = GridSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(x_train, y_train)
print(gs.best_score_, gs.best_params_)

521/521 [==============================] - 1s 2ms/step
0.3348198810654772 {'hidden_layer_dim': 100, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam', 'optimizer__learning_rate': 0.001}


In [22]:
def get_model(hidden_layer_dim, meta, layer_cnt):

    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]

    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(32, 32, 3)))

    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))

    for i in range(layer_cnt):
      model.add(keras.layers.Dense(hidden_layer_dim))
      model.add(keras.layers.Activation("relu"))

    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

In [23]:
clf = KerasClassifier(
    get_model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
    layer_cnt=1,
)

In [24]:
params = {
    #"hidden_layer_dim": [50, 100, 200],
    "hidden_layer_dim": [100],
    "loss": ["sparse_categorical_crossentropy"],
    #"optimizer": ["adam", "sgd"],
    "optimizer": ["adam"],
    #"optimizer__learning_rate": [0.0001, 0.001, 0.1],
    "optimizer__learning_rate": [0.001],
    "layer_cnt": [1, 2, 3, 4, 5],
}
gs = GridSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(x_train, y_train)
print(gs.best_score_, gs.best_params_)

521/521 [==============================] - 2s 3ms/step
0.33346002226721333 {'hidden_layer_dim': 100, 'layer_cnt': 4, 'loss': 'sparse_categorical_crossentropy', 'optimizer': 'adam', 'optimizer__learning_rate': 0.001}


In [5]:
!pip install optuna

In [6]:
import optuna
from tensorflow.keras.optimizers import Adam, SGD

In [40]:
def objective(trial):
    units = trial.suggest_categorical('units', [32, 64, 128])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    layer_cnt = trial.suggest_categorical('layer_cnt', [1, 2, 3, 4, 5 ])

    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(32, 32, 3)))
    for i in range(layer_cnt):
      model.add(keras.layers.Dense(units, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]

    return val_accuracy

In [41]:
# Create a study object and optimize
study = optuna.create_study(direction='maximize')  # 'maximize' for accuracy, 'minimize' for loss
study.optimize(objective, n_trials=10)

# Print the best parameters and value
print("Best parameters found: ", study.best_params)
print("Best value found: {:.2f}%".format(study.best_value * 100))

[I 2023-09-03 11:32:52,884] A new study created in memory with name: no-name-3012dad4-ca38-4d13-a6eb-1bad15f78532
[I 2023-09-03 11:35:17,831] Trial 0 finished with value: 0.38589999079704285 and parameters: {'units': 64, 'learning_rate': 0.0023403926577720393, 'layer_cnt': 5}. Best is trial 0 with value: 0.38589999079704285.
[I 2023-09-03 11:36:42,606] Trial 1 finished with value: 0.3919000029563904 and parameters: {'units': 32, 'learning_rate': 0.00048807528794141255, 'layer_cnt': 4}. Best is trial 1 with value: 0.3919000029563904.
[I 2023-09-03 11:38:46,778] Trial 2 finished with value: 0.4794999957084656 and parameters: {'units': 128, 'learning_rate': 0.00013264561402152812, 'layer_cnt': 1}. Best is trial 2 with value: 0.4794999957084656.
[I 2023-09-03 11:41:11,247] Trial 3 finished with value: 0.46970000863075256 and parameters: {'units': 64, 'learning_rate': 0.000715465195981452, 'layer_cnt': 3}. Best is trial 2 with value: 0.4794999957084656.
[I 2023-09-03 11:42:21,686] Trial 4 f

Best parameters found:  {'units': 128, 'learning_rate': 0.00013264561402152812, 'layer_cnt': 1}
Best value found: 47.95%


In [7]:
def objective(trial):
    units = trial.suggest_categorical('units', [32, 64, 128])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    #layer_cnt = trial.suggest_categorical('layer_cnt', [1, 2, 3, 4, 5 ])

    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(32, 32, 3)))
    model.add(keras.layers.Dense(units, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), verbose=0)
    val_accuracy = history.history['val_accuracy'][-1]

    return val_accuracy

In [8]:
# Create a study object and optimize
study = optuna.create_study(direction='maximize')  # 'maximize' for accuracy, 'minimize' for loss
study.optimize(objective, n_trials=10)

# Print the best parameters and value
print("Best parameters found: ", study.best_params)
print("Best value found: {:.2f}%".format(study.best_value * 100))

[I 2023-09-03 12:24:57,084] A new study created in memory with name: no-name-65e4a804-5307-4c34-9e76-43af8d359f2f
[I 2023-09-03 12:26:25,434] Trial 0 finished with value: 0.44449999928474426 and parameters: {'units': 64, 'learning_rate': 0.002578919112574161}. Best is trial 0 with value: 0.44449999928474426.
[I 2023-09-03 12:27:25,200] Trial 1 finished with value: 0.23100000619888306 and parameters: {'units': 32, 'learning_rate': 3.987078314562969e-05}. Best is trial 0 with value: 0.44449999928474426.
[I 2023-09-03 12:28:25,181] Trial 2 finished with value: 0.3059000074863434 and parameters: {'units': 64, 'learning_rate': 0.00011969635163942016}. Best is trial 0 with value: 0.44449999928474426.
[I 2023-09-03 12:29:49,456] Trial 3 finished with value: 0.3528999984264374 and parameters: {'units': 64, 'learning_rate': 0.00024029478579775556}. Best is trial 0 with value: 0.44449999928474426.
[I 2023-09-03 12:30:51,144] Trial 4 finished with value: 0.24789999425411224 and parameters: {'unit

Best parameters found:  {'units': 64, 'learning_rate': 0.002578919112574161}
Best value found: 44.45%
